In [1]:
import inline as inline
import matplotlib as matplotlib
from math import log
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from konlpy.tag import Okt
from collections import Counter
import re
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

path = 'C:/Users/kimdm/working_volume/data_warehouse/keyword/sample/sample_data' #데이터 들어있는 경로로 지정해주세요(데이터 외 다른 파일 없게 해주셔야해요)
forders = os.listdir(path)
print(forders)

df_all = pd.DataFrame()
for i in range(0, len(forders)):
    if forders[i].split('.')[1] == 'csv':
        s_file = 'C:/Users/kimdm/working_volume/data_warehouse/keyword/sample/sample_data/' + forders[i]  # 'sample/' 부분을 '데이터가 들어있는 폴더명/'으로 변경해주세요
        df = pd.read_csv(s_file, encoding='cp949')
        basename = os.path.basename(forders[i])
        file_name = os.path.splitext(basename)[0]
        df.insert(0, 'subject', file_name)
        df_all = pd.concat([df_all, df])
        print(df_all)


review = df_all['1'].values


['SeoulTech혜윰강좌박종열.csv', '경제의이해이상욱.csv', '과학기술과사회김명심.csv', '과학기술과사회김희윤.csv', '교통공학송기한.csv', '동역학김대현.csv', '법학개론강기홍.csv', '사회의이해조명희.csv', '삶의윤리학우상권.csv', '수리학박인환.csv', '응용재료역학허남수.csv', '정치의이해장명학.csv', '취업과진로김효순.csv', '측량정보공학및실습이창노.csv', '토질역학(1)박헌준.csv', '한국사의재조명이정훈.csv', '현대문화론최진호.csv', '현대사회와철학이수정.csv', '확률과통계이한청.csv', '환경과인간정연훈.csv', '회계학개론전중열.csv']
              subject                                                  1
0    SeoulTech혜윰강좌박종열                      개 꿀 강 좌. 안들으면 손해. 이런 꿀교양이 없다.
1    SeoulTech혜윰강좌박종열                       할 말 없어요.. 그냥 무조건 따지지 말고 들으세요
2    SeoulTech혜윰강좌박종열                   온라인강의에 패논패과목이라 부담적고 시험안보는거 진짜 꿀강
3    SeoulTech혜윰강좌박종열                       개꿀강의. 부담없이 들을 수 있는 학점채우기용 과목
4    SeoulTech혜윰강좌박종열               녹강으로 진행되구 패논패에 시험도 감상문으로 대체라 최고의 과목!
..                ...                                                ...
150  SeoulTech혜윰강좌박종열             개꿀과목\n학점채우고 싶을때 들으면 됨\n과제도 보고서 두번이 끝!!
151  SeoulTech혜윰강좌박종열            중간 기말 다 감상문 작성이었고 인공지능에 관한 수업

In [ ]:
def apply_regular_expression(review):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', review)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result

okt = Okt()

stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
#나중에 우리 데이터만의 특화된 불용어 있으면 추가하기


def text_cleaning(review): #데이터 정제
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')
    result = hangul.sub('', review)
    okt = Okt()  # 형태소 추출
    nouns = okt.nouns(result)
    nouns = [x for x in nouns if len(x) > 1]  # 한글자 키워드 제거
    nouns = [x for x in nouns if x not in stopwords]  # 불용어 제거
    return nouns

vect = CountVectorizer(tokenizer = lambda x: text_cleaning(x))
bow_vect = vect.fit_transform(review.tolist())
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)
word_count_dict = dict(zip(word_list, count_list))

tfidf_vectorizer = TfidfTransformer() #tf-idf 구하기
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)
ivert_index_vectorizer = {v:k for k, v in vect.vocabulary_.items()}



print(tf_idf_vect)
#print하면 (x,y) 0.xxxx 이런 식으로 나올텐데 x는 리뷰, y는 해당 단어 코드 같은거라고 생각하시면 됩니다
# 0.xxxx는 tf_idf 값입니다
print(ivert_index_vectorizer) #단어 코드에 해당하는 단어를 알고싶을때 사용